# Llama-Index Async

Async is a growing method for LLM applications and can be especially useful for reducing indexing time. This notebook demonstrates how to monitor Llama-index async apps with TruLens.

### Import from Llama-Index and TruLens

In [ ]:
# ! pip install trulens_eval==0.9.0 llama_index== 0.7.16 html2text==2020.1.16

In [ ]:
from llama_index import VectorStoreIndex, SimpleWebPageReader
import openai
from trulens_eval import TruLlama, Feedback, Tru, feedback, Select

tru = Tru()

### Add API keys
For this example you need an OpenAI key

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."
openai.api_key = os.environ["OPENAI_API_KEY"] 

### Create Async App

In [ ]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_chat_engine(streaming=True)

In [ ]:
response = await query_engine.achat("What did the author do growing up?")

print(response)

### Set up Evaluation

In [ ]:
import numpy as np

# Initialize OpenAI-based feedback function collection class:
openai = feedback.OpenAI()

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance, name="QA Relevance").on_input_output()

### Create tracked app

In [ ]:
tru_query_engine = TruLlama(
    query_engine, feedbacks=[f_qa_relevance, f_qs_relevance]
)

### Run Async Application with TruLens

In [ ]:
# Instrumented query engine can operate like the original:
llm_response_async, rec_async = await tru_query_engine.aquery_with_record("What did the author do growing up?")

print(llm_response_async)

In [ ]:
tru.run_dashboard()